https://www.psycopg.org/docs/usage.html#the-problem-with-the-query-parameters

https://www.psycopg.org/docs/sql.html
psycopg2.sql – SQL string composition¶
New in version 2.7.

In [2]:
import psycopg2
import os, getpass
from psycopg2.sql import SQL, Identifier


In [3]:
def create_server_connection(database, user):
    conn = None
    try:
        conn = psycopg2.connect(
            database=database,
            user=user,
        )
        print(f"Connected to Database {database} successfully")
        
    except Exception as e:
        print(e)

    return conn

In [4]:
def create_or_drop_database(conn, query):
    autocommit = conn.autocommit #preserve the value of autocommit
    conn.autocommit = True #MUST BE set to True since Postgresql cannot create a database in a transaction
    with conn.cursor() as cur:
        try:
            cur.execute(query)
            print(f"Query: {query} ran successfully.")
        except Exception as e:
            print(e)
        finally:
            #be a good citizen and put stuff back the way you found it
            conn.autocommit = autocommit 

In [5]:
#version 2
def execute_query(conn, query):
    autocommit = conn.autocommit #preserve the value of autocommit
    conn.autocommit = True #MUST BE set to True since Postgresql cannot create a database in a transaction
    cur = conn.cursor()
    try:
        cur.execute(query)
        print(f"Query: {query}ran successfully.")
    except Exception as e:
        print(e)
    finally:
        #be a good citizen and put stuff back the way you found it
        conn.autocommit = autocommit 
    return cur

In [6]:
db, user = "jupyterdb", getpass.getuser()
conn = create_server_connection(db,user)

Connected to Database jupyterdb successfully


In [7]:
conn.autocommit = True

In [8]:
# let's create an authors table in our database

In [9]:
query = """
DROP TABLE IF EXISTS author;

CREATE TABLE author(
name text,
email text
);
"""
cur = execute_query(conn, query)
cur.close()

Query: 
DROP TABLE IF EXISTS author;

CREATE TABLE author(
name text,
email text
);
ran successfully.


In [10]:
query = "INSERT INTO author (name) VALUES (%s);"
data = ("me", ) # gotcha: the data argument which is substituted into %s *MUST* BE AN ITERABLE 
cur = conn.cursor()
cur.execute(query, data)
cur.close()

NOTE: THE ONLY FORMAT STRING IS %s 
    
IT WORKS FOR ALL DATA TYPES!!!!!!!!!!!!!!!!!!!

In [13]:
query = "SELECT * FROM author;"
cur = conn.cursor()
cur.execute(query)
print(cur.fetchall())
cur.close()

[('me', None)]


In [14]:
query = "INSERT INTO author (name, email) VALUES (%s, %s);"
data = ("John Donne", "death@benotproud.com")
cur = conn.cursor()
cur.execute(query, data)
cur.close()

In [15]:
query = "SELECT * FROM author;"
cur = conn.cursor()
cur.execute(query)
print(cur.fetchall())
cur.close()

[('me', None), ('John Donne', 'death@benotproud.com')]


In [16]:
query = "INSERT INTO author (name, email) VALUES (%s, %s), (%s,%s);"
data = ["Octavia Butler", "kindred@thefutureisnow.com", "Alfred Neuman", "whatmeworry@worryfree.com"] #any iterable is fine
cur = conn.cursor()
cur.execute(query, data)
cur.close()

In [17]:
query = "SELECT * FROM author;"
cur = conn.cursor()
cur.execute(query)
print(cur.fetchall())
cur.close()

[('me', None), ('John Donne', 'death@benotproud.com'), ('Octavia Butler', 'kindred@thefutureisnow.com'), ('Alfred Neuman', 'whatmeworry@worryfree.com')]


In [ ]:
#now let's change our execute_query to use data in the second argument of cur.execute()

In [ ]:
#version 4
def execute_query(conn, *query):
    try:
        autocommit = conn.autocommit #preserve the value of autocommit
        conn.autocommit = True
        cur = conn.cursor()
        
        if len(query) == 2:
            cur.execute(*query)
            print(f"Query: {query[0]} with data:{query[1]} ran successfully.")
        else:
            cur.execute(query[0])
            print(f"Query: {query[0]} ran successfully.")
    
    except Exception as e:
        print(e)
    finally:
        #put stuff back the way you found it
        conn.autocommit = autocommit 
    return cur

In [ ]:
query = "INSERT INTO author (name) VALUES (%s);"
data = ("O'Reilly", )
cur = execute_query(conn, query, data)
cur.close()

psycopg2.sql – SQL string composition

The module contains objects and functions useful to generate SQL dynamically, in a convenient and safe way. SQL identifiers (e.g. names of tables and fields) cannot be passed to the execute() method like query arguments:

In [ ]:
# This will not work
cur = conn.cursor()
table_name = 'my_table'
cur.execute("insert into %s values (%s, %s)", [table_name, 10, 20])
cur.close()

In [ ]:
# This is the correct way

from psycopg2 import sql
cur = conn.cursor()

cur.execute(sql.SQL("INSERT INTO {} VALUES (%s)").format(sql.Identifier('author')),["herman melville"])

cur.close()

1) use the sql.SQL() function, just like you'd use str.format()
2) inside the argument of format, pass the table name inside sql.Identifier
3) pass the data in the second argument of cur.execute


In [ ]:
sql.Identifier('author')

In [ ]:
sql.SQL("insert into {} values (%s)").format(sql.Identifier('author'))

In [ ]:
# gotcha: the data argument which is substituted into %s *MUST* BE AN ITERABLE 


data = ['herman melville'] in this case

cur.execute(sql.SQL("INSERT INTO {} VALUES (%s)").format(sql.Identifier('author')),
            ["herman melville"]
           )

In [ ]:
cur.execute(sql.SQL("INSERT INTO {} VALUES (%s)").format(sql.Identifier('author')), ["herman melville"]
           )

In [ ]:
cur = conn.cursor()

cur.execute(sql.SQL("INSERT INTO {} VALUES (%s, %s)").format(sql.Identifier('author')),["edgar allan poe","raven1845@gmail.com" ])

cur.close()

In [ ]:
db, user = "jupyterdb", getpass.getuser()
conn = create_server_connection(db,user)

In [ ]:
query = """
DROP TABLE IF EXISTS author;

CREATE TABLE author(
name text,
email text
);
"""
cur = execute_query(conn, query)
cur.close()

In [ ]:
query, data = sql.SQL("INSERT INTO {} VALUES (%s, %s)").format(sql.Identifier('author')), ["edgar allan poe","raven1845@gmail.com" ]
cur = execute_query(conn, query, data)
cur.close()

In [ ]:
query = "SELECT * FROM author;"
cur = execute_query(conn, query)
cur.fetchall()
cur.close()